In [32]:
#using the last iteration of tfid_explore, make a df of all tfid vectorized POASTS with the label of 1 or 0 to run through model
#we'll train model on what is or isn't a troll piece of CONTent, then aggregate the amount of troll content in a users post history.
#old models have been running the FIRST pice of content only ("tfid_vector[0] ONLY, not the whole corpus")
#we have been converting a user into a single row of one post instead of converting a user into an entire ARRAY

In [33]:
from _keys import db_user, db_password, db_name, db_host, db_port
import psycopg2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
stop=set(stopwords.words('english'))

In [34]:
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
sql = """
select *
	from (
		select author, title, 1 as is_bot, id
		from sus_user_posts
		where author in (select distinct author from sus_user_posts) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		)as posts_aggregate
union
select *
	from (
		select author, title, 0 as is_bot, id
		from norm_user_posts
		where author in (select distinct author from norm_user_posts limit 1500) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		)as norm_agg
"""

In [35]:
cur = conn.cursor()
cur.execute(sql)
output = cur.fetchall()

In [36]:
authors = []
posts = []
bot_status = []
post_ids = []
for i in range(len(output)):
    authors.append(output[i][0])
    posts.append(output[i][1])
    bot_status.append(output[i][2])
    post_ids.append(output[i][3])

In [37]:
cv = CountVectorizer(min_df=1)
word_count_vector = cv.fit_transform(posts)
tfid_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfid_transformer.fit(word_count_vector)
tf_idf_vectors = tfid_transformer.transform(word_count_vector)
feature_names = cv.get_feature_names_out()
df = pd.DataFrame(tf_idf_vectors.T.todense(), index=feature_names, columns=post_ids)
df = df.T
df['___author'] = authors
df['___bot_status'] = bot_status

In [38]:
df = df.reset_index()
df = df.rename(columns={'level_0':'post_id'})

In [39]:
df.to_csv('tfid_post_vectorize_agg.csv', index=False)

In [40]:
df.head()

,post_id,00,000,000th,0079,01,02,030,039,04,...,पह,নত,არაბული,თამთა,თემო,ოდიკაძე,ონისე,蔡小煒,___author,___bot_status
0,3brkjs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
1,32w130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
2,31c3d7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,_____goats,0
3,3em4to,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,__8ball__,0
4,3coa1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,__8ball__,0
